In [1]:
import numpy as np
import tensorflow as tf

2022-05-12 10:36:52.347662: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [67]:
N = 10
xt, yt = np.arange(N), np.arange(10)

In [68]:
xt

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [69]:
ds = tf.data.Dataset.from_tensor_slices(xt)
list(ds.as_numpy_iterator())
len(ds)

10

In [70]:
from typing import Tuple

In [113]:
class DS(tf.keras.utils.Sequence):
    def __init__(self, name: str, n: int, b: int):
        self.name = name
        self.n = n
        self.b = b
        self.x = np.random.uniform(size=(self.n, 10, 10, 3))
        self.y = np.random.randint(0, 2, size=(self.n, 10))
    
    def __len__(self) -> int:
        return self.n // self.b + self.n % self.b

    def __getitem__(self, ind: int) -> Tuple[np.ndarray]:
        inds = slice(self.b * ind, self.b * (ind + 1))
        x, y = self.x[inds], self.y[inds]
        print(f'DS({self.name}).__getitem__', ind, len(x))
        return self.x[inds], self.y[inds]
    
    def on_epoch_end(self):
        print(f'DS({self.name}).on_epoch_end')
        
    def gen(self):
        nb = self.n // self.b + self.n % self.b
        ib = 0
        while True:
            inds = slice(self.b * ib, self.b * (ib + 1))
            x, y = self.x[inds], self.y[inds]
            print(f'DS({self.name})', ib, len(x))
            yield x, y
            ib += 1
            if ib == nb:
                print(f'DS({self.name}) is over')
                ib = 0


In [114]:
def get_uncompiled_model():
    inp = tf.keras.Input(shape=(10, 10, 3), dtype=tf.float32)
    x = inp
    x = tf.keras.layers.Conv2D(5, 3, activation='relu')(x)
    x = tf.keras.layers.Conv2D(10, 3, activation='relu')(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Softmax()(x)
    out = x
    model = tf.keras.models.Model(inputs=inp, outputs=out)
    return model


In [115]:
model = get_uncompiled_model()
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-3),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.CategoricalAccuracy()],
)

In [116]:
ds_train = DS('train', 20, 4)
ds_val = DS('val', 10, 4)

In [104]:
model.fit(ds_train, epochs=3, steps_per_epoch=2, validation_data=ds_val, validation_steps=2,
          max_queue_size=0, workers=0, shuffle=True)

DS(train).__getitem__ 0 4
Epoch 1/3
DS(train).__getitem__ 0 4
1/2 [==============>...............] - ETA: 0s - loss: 14.2495 - categorical_accuracy: 0.0000e+00DS(train).__getitem__ 1 4
DS(val).__getitem__ 0 4
DS(val).__getitem__ 0 4
DS(val).__getitem__ 1 4
DS(val).on_epoch_end
2/2 [==============================] - 0s 63ms/step - loss: 12.3325 - categorical_accuracy: 0.1250 - val_loss: 11.9955 - val_categorical_accuracy: 0.1250
DS(train).on_epoch_end
Epoch 2/3
DS(train).__getitem__ 0 4
1/2 [==============>...............] - ETA: 0s - loss: 14.3490 - categorical_accuracy: 0.0000e+00DS(train).__getitem__ 1 4
DS(val).__getitem__ 0 4
DS(val).__getitem__ 1 4
DS(val).on_epoch_end
2/2 [==============================] - 0s 43ms/step - loss: 12.4135 - categorical_accuracy: 0.1250 - val_loss: 12.0610 - val_categorical_accuracy: 0.1250
DS(train).on_epoch_end
Epoch 3/3
DS(train).__getitem__ 0 4
1/2 [==============>...............] - ETA: 0s - loss: 14.4533 - categorical_accuracy: 0.0000e+00DS(trai

In [117]:
out_sig = (
        tf.TensorSpec(shape=(None, 10, 10, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 10), dtype=tf.float32),
    )
ds_train_gen = tf.data.Dataset.from_generator(ds_train.gen, output_signature=out_sig)
ds_val_gen = tf.data.Dataset.from_generator(ds_val.gen, output_signature=out_sig)
model.fit(ds_train_gen, epochs=3, steps_per_epoch=2, validation_data=ds_val_gen, validation_steps=2,
          max_queue_size=0, workers=0, shuffle=True)

Epoch 1/3
DS(train) 0 4
1/2 [==============>...............] - ETA: 0s - loss: 12.1399 - categorical_accuracy: 0.2500DS(train) 1 4
DS(val) 0 4
DS(val) 1 4
2/2 [==============================] - 0s 149ms/step - loss: 11.1460 - categorical_accuracy: 0.1667 - val_loss: 10.9498 - val_categorical_accuracy: 0.0000e+00
Epoch 2/3
DS(train) 2 4
1/2 [==============>...............] - ETA: 0s - loss: 10.4354 - categorical_accuracy: 0.0000e+00DS(train) 3 4
DS(val) 0 4
DS(val) 1 4
2/2 [==============================] - 0s 34ms/step - loss: 10.8114 - categorical_accuracy: 0.0000e+00 - val_loss: 10.9561 - val_categorical_accuracy: 0.0000e+00
Epoch 3/3
DS(train) 4 4
1/2 [==============>...............] - ETA: 0s - loss: 11.4056 - categorical_accuracy: 0.2500DS(train) is over
DS(train) 0 4
DS(val) 0 4
DS(val) 1 4
2/2 [==============================] - 0s 38ms/step - loss: 11.6635 - categorical_accuracy: 0.2500 - val_loss: 10.9698 - val_categorical_accuracy: 0.0000e+00


In [20]:
np.random.uniform(0, 2, size=(2, 3)).astype(int)

array([[0, 0, 0],
       [0, 1, 1]])

In [31]:
n = 8
b = 4

In [32]:
n // b, n % b

(2, 0)

In [33]:
l = np.arange(7)

In [34]:
l

array([0, 1, 2, 3, 4, 5, 6])

In [35]:
l[2:3]

array([2])

In [40]:
l[slice(4, 7)]

array([4, 5, 6])

In [3]:
y_true = [1, 2]
y_pred = [[0.05, 0.95, 0], [0.1, 0.8, 0.1]]
# Using 'auto'/'sum_over_batch_size' reduction type.
scce = tf.keras.losses.SparseCategoricalCrossentropy()
scce(y_true, y_pred).numpy()

1.1769392

In [4]:
yt = np.array(y_true)[..., None]
yp = np.array(y_pred)
print(yt)
print(yp)
print(scce(yt, yp))

[[1]
 [2]]
[[0.05 0.95 0.  ]
 [0.1  0.8  0.1 ]]
tf.Tensor(1.1769392490386963, shape=(), dtype=float64)
